# Getting started with HTMD Molecules

Assuming that you have already downloaded and installed **HTMD**, this tutorial introduces you to the software, specially into the `Molecule` class, whose features serve as a good introduction for the more complex features of HTMD.

Let's get started! The first thing you will have to get familiar with in HTMD is the [Molecule](https://www.htmd.org/docs/latest/htmd.molecule.molecule.html) class.
Objects of this class are meant to *store structural information on molecules*.
Despite it's name, a `Molecule` object does not only contain a single molecule. 
Instead, similar to [VMD](http://www.ks.uiuc.edu/Research/vmd/) (a visualization software we also use in HTMD), a `Molecule` object can also contain a whole system including water, ions, proteins, ligands, lipids etc. 
So think of it as a container of structural information.

First, we need to import HTMD, so that any class and function defined by HTMD is available in the workspace:

In [1]:
from htmd import *

Videos from the HTMD2015 workshops are available on the Acellera youtube channel: https://www.youtube.com/user/acelleralive

You are on the latest HTMD version (unpackaged).


## Reading files

The [Molecule](https://www.htmd.org/docs/latest/htmd.molecule.molecule.html) class provides file readers for various structure formats like PDB, PRMTOP, PSF, GRO, MOL2, MAE and more. It is also able to read various MD trajectory and coordinate formats including XTC, DCD, COOR, CRD, TRR, XYZ etc. If in doubt just try to read a file and it will tell you if it is able to read the given format. 
The method for reading files is [Molecule.read()](https://www.htmd.org/docs/latest/htmd.molecule.molecule.html#htmd.molecule.molecule.Molecule.read), however you can also specify the file name in the class constructor and it will automatically call `read()`.
Let's see an example.

In [ ]:
mol = Molecule('3PTB')  # This downloads a PDB file directly from the RCSB server

or just use a local file:

    mol = Molecule('yourprotein.pdb')

PDB files contain both atom information and coordinates. Some other formats separate the atom information from the coordinates. 
In that case you can start for example by reading atom information from a PSF file and then read atom coordinates using the `read` method of `Molecule` as in the next example.
You could also read them in reverse order, creating the `Molecule` using the XTC and then reading the PSF (it would not matter).

    mol = Molecule('yourstructure.psf')
    mol.read('yourtrajectory.xtc')

## Writing files

The `Molecule` class also provides file writers for multiple formats using the [Molecule.write()](https://www.htmd.org/docs/latest/htmd.molecule.molecule.html#htmd.molecule.molecule.Molecule.write) method.

    mol.write('yourtrajectory.dcd')
    mol.write('yourstructure.prmtop')

## Looking inside a Molecule

Printing the `Molecule` object shows its properties:

In [3]:
print(mol)

Molecule with 1701 atoms and 1 frames
PDB field - altloc shape: (1701,)
PDB field - beta shape: (1701,)
PDB field - chain shape: (1701,)
PDB field - charge shape: (1701,)
PDB field - coords shape: (1701, 3, 1)
PDB field - element shape: (1701,)
PDB field - insertion shape: (1701,)
PDB field - name shape: (1701,)
PDB field - occupancy shape: (1701,)
PDB field - record shape: (1701,)
PDB field - resid shape: (1701,)
PDB field - resname shape: (1701,)
PDB field - segid shape: (1701,)
PDB field - serial shape: (1701,)
bonds shape: (42, 2)
box shape: (3, 1)
fileloc shape: (1, 2)
frame: 0
masses shape: (1701,)
reps: 
ssbonds shape: (0,)
step shape: (0,)
time shape: (0,)
topoloc: /home/toni/work/htmd/htmd/tutorials/3PTB
viewname: 3PTB


## Properties and methods of `Molecule` objects

Each `Molecule` object has a number of **properties** (data associated to the molecule) and **methods** (operations that you can perform on the molecule). Some of the properties correspond to data which is usually found in PDB files.

| Properties |Methods      |
|:----------:|:-----------:|
|record      |read( )      |
|serial      |write( )     |
|name        |get( )       |
|resname     |set( )       |
|chain       |atomselect( )|
|resid       |copy( )      |
|segid       |filter( )    |
|coords      |append( )    |
|box         |insert( )    |
|reps        |view( )      |
|...         |moveBy( )    |
|            |rotateBy( )  |
|            |...          |

Properties can be accessed,

* either directly:

In [4]:
mol.serial

array([   1,    2,    3, ..., 1700, 1701, 1702])

or,

* via the `Molecule.get` method:

In [5]:
mol.get('serial')

array([   1,    2,    3, ..., 1700, 1701, 1702])

Similarly, they can be modified directly, or via the `Molecule.set` method. This pair of methods is known as "getter/setter" methods in the object-oriented jargon. **The following sections will show the usage of property getters and setters in a number of real-world tasks.**

### Check the residue IDs of Cysteines in the `Molecule`

In order to get the residue IDs of cysteines in the molecule, one can do:

In [6]:
mol.get('resid', sel='resname CYS')

array([ 22,  22,  22,  22,  22,  22,  42,  42,  42,  42,  42,  42,  58,
        58,  58,  58,  58,  58, 128, 128, 128, 128, 128, 128, 136, 136,
       136, 136, 136, 136, 157, 157, 157, 157, 157, 157, 168, 168, 168,
       168, 168, 168, 182, 182, 182, 182, 182, 182, 191, 191, 191, 191,
       191, 191, 201, 201, 201, 201, 201, 201, 220, 220, 220, 220, 220,
       220, 232, 232, 232, 232, 232, 232])

**Note:** residue IDs are outputted multiple times. This is due to the fact that one value is returned per matched atom, and there are 6 atoms resolved per Cys residue.

The names of the 6 atoms of cysteine residue 58 can be checked with:

In [7]:
mol.get('name','resname CYS and resid 58')

array(['N', 'CA', 'C', 'O', 'CB', 'SG'], dtype=object)

To obtain one residue ID per residue, one can either,

* further restrict the selection to carbon &alpha; atoms:

In [8]:
mol.get('resid',sel='name CA and resname CYS')

array([ 22,  42,  58, 128, 136, 157, 168, 182, 191, 201, 220, 232])

or,

* take advantage of `python` and use the `numpy.unique` function to remove repeated entries:

In [9]:
np.unique(mol.get('resid',sel='resname CYS'))

array([ 22,  42,  58, 128, 136, 157, 168, 182, 191, 201, 220, 232])

**Note:** `numpy` is imported as `np` when `htmd` is imported.

### Retrieve the coordinates of atoms

This is done by accessing the `Molecule.coords` property. It is a special property, since it returns a 3-column vector (for the three coordinates).

In [10]:
mol.get('coords','resname CYS and resid 58 and name CA')

array([  4.23999977,  16.49500084,  27.98600006], dtype=float32)

**Note**: the precision is restricted to the one in the PDB file.

If more than one atom (say, N>1) are selected, an `N x 3` matrix is returned:

In [11]:
mol.get('coords','resname CYS and resid 58')

array([[  5.12200022,  16.71899986,  26.86300087],
       [  4.23999977,  16.49500084,  27.98600006],
       [  4.87400007,  16.95800018,  29.29999924],
       [  4.23799992,  16.76399994,  30.36199951],
       [  3.94099998,  14.9989996 ,  28.07099915],
       [  2.79200006,  14.45199966,  26.72200012]], dtype=float32)

### Know the chains or segments present in the `Molecule`

The chains present in the `Molecule` can be known using:

In [12]:
np.unique(mol.get('chain'))

array(['A'], dtype=object)

which means that every atom is assigned to the same chain (i.e., chain A).

The same can be done for segments:

In [13]:
np.unique(mol.get('segid'))

array(['0', '1'], dtype=object)

These segment IDs may need to be changed for system building.

### Set properties of the `Molecule`

`Molecule.set` can be used to change/name/rename specific fields of the molecule or a selection.

For example, it can create a segment ID called 'P' for all protein atoms:

In [14]:
mol.set('segid','P','protein')
np.unique(mol.get('segid', 'protein'))

array(['P'], dtype=object)

Another useful task for `Molecule.set` is to rename residues.

For example, renaming all HIS residues to 'HSN':

In [15]:
mol.set('resname','HSN','resname HIS')
mol.get('serial',sel='resname HSN')

array([159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 283, 284, 285,
       286, 287, 288, 289, 290, 291, 292, 537, 538, 539, 540, 541, 542,
       543, 544, 545, 546])

### Count the number of waters in the `Molecule`

Get the indices of atoms that were recognized as water:

In [16]:
mol.get("serial",sel="water")

array([1641, 1642, 1643, 1644, 1645, 1646, 1647, 1648, 1649, 1650, 1651,
       1652, 1653, 1654, 1655, 1656, 1657, 1658, 1659, 1660, 1661, 1662,
       1663, 1664, 1665, 1666, 1667, 1668, 1669, 1670, 1671, 1672, 1673,
       1674, 1675, 1676, 1677, 1678, 1679, 1680, 1681, 1682, 1683, 1684,
       1685, 1686, 1687, 1688, 1689, 1690, 1691, 1692, 1693, 1694, 1695,
       1696, 1697, 1698, 1699, 1700, 1701, 1702])

**Note:** in this case, the hydrogens of waters are not present, so we only get one index per water without using the `np.unique` function.

Then, the number of waters present in the `Molecule` can be obtained with:

In [17]:
len(mol.get("serial",sel="water"))

62

Alternatively, the `Molecule.atomselect` method can be used. It returns a vector of boolean values:

In [18]:
mol.atomselect("water")

array([False, False, False, ...,  True,  True,  True], dtype=bool)

The fact that `True` counts as 1 in the `sum` function can be used to obtain, through a different way, the number of waters:

In [19]:
selection = mol.atomselect("water")
print(selection)
sum(selection)

[False False False ...,  True  True  True]


62

## Representations and Visualization

`Molecule` objects can be visualized either in VMD or in NGL, a WebGL javascript molecule viewer that's integrated in the Notebook.

By default, VMD is used as a viewer. If one wants to use NGL, run the following:

In [20]:
htmd.config(viewer='webgl')

### On-the-fly molecule visualization and interaction

In [21]:
mol = Molecule('3PTB') # reloading the molecule
mol.view()

It is also possible to apply multiple representations to a `Molecule` as in VMD.

Representations use the same names as in VMD, even when using the NGL viewer. Important parameters are: **style**, **color**, and **sel**.   

### The "quick" or "transient" view

Use the `Molecule.view` method, specifying the representation as arguments. Use the `hold` parameter so that following `Molecule.view` calls can overlay. Otherwise, representations will be cleared on every call.

In [22]:
mol.view(sel='protein', style='NewCartoon', color='Index', hold=True)
mol.view(sel='resname BEN', style='Licorice', color=1)

### The "explicit" way: using `Molecule.reps`

One directly manipulates elements in the `reps` property of `Molecule` objects, with the views being stored in that property.

In [23]:
mol.reps.remove()   # Clear representations
mol.reps.add(sel='protein', style='NewCartoon', color='Index')
mol.reps.add(sel='resname BEN', style='Licorice', color=1)
print(mol.reps)     # Show list of representations (equivalent to mol.reps.list())
mol.view()

rep 0: sel='protein', style='NewCartoon', color='Index'
rep 1: sel='resname BEN', style='Licorice', color='1'



### Atom selection expressions work as in VMD

The following shows the molecule without a 6 &Aring; thick slab in the middle (&minus;3 &Aring; &le; `x` &le; +3 &Aring;):

In [24]:
mol.reps.remove() # in order to remove the previouly stored representations
mol.view(sel='x*x>9')

## Duplicate, modify and write objects

Use `Molecule.copy` to duplicate the molecule into a new object:

In [25]:
newmol = mol.copy()

`Molecule.write` can be used to output a PDB file of your whole molecule (or just a selection). The following command uses the above copied molecule `newmol` to write out a PDB file of the benzamidine ligand atoms present in the fetched PDB file, except for hydrogen atoms:

In [26]:
newmol.write('/tmp/ligand.pdb','resname BEN and noh')

`Molecule.filter` can be used to select specific parts of the molecule (e.g. chains, segments) and clean/remove all the rest.

For example, clean all except for protein atoms in chain A:

In [27]:
mol.filter('chain A and protein')
mol.view()

## Joining molecules/segments

`Molecule.append` appends a Molecule object (e.g. ligand, water or ion segments, etc.) to another molecule object (e.g. protein, membrane).

For example, to append the benzamidine ligand (saved above as a PDB file) to the molecule we are working with (which is now only the protein chain A), simply do:

In [28]:
ligand = Molecule('/tmp/ligand.pdb')
mol.append(ligand)
mol.view()

You can also add an atom using `Molecule.insert`:

In [29]:
atom = Molecule()
atom.record = 'ATOM'
atom.name = 'CA'
atom.resid = 0
atom.resname = 'XXX'
atom.chain = 'X'
atom.coords = [6, 3, 2]

In [30]:
newligand = ligand.copy()
newligand.insert(atom, 0)
newligand.view(style='VDW')

## Playing with coordinates

Coordinates can be used to perform geometric tasks on your molecule, such as translations or rotations.

For example, calculate the geometric center of your molecule:

In [31]:
coo = mol.get('coords')
c = np.mean(coo, axis=0)

Then, `Molecule.moveBy` can be used to translate the molecule and center it on the origin [0, 0, 0] using the previosly calculated center `c`:

In [32]:
mol.moveBy(-c)

Finally, check the new center:

In [33]:
np.mean(mol.get('coords'),axis=0)

array([  2.15420940e-07,  -2.36497249e-06,   4.63504330e-06], dtype=float32)

## Rotations
Another common task is rotations (e.g. to build a protein - ligand system).
For example, load up your ligand and visualize its orientation:

In [34]:
ligand = Molecule('3PTB')
ligand.filter('resname BEN')
ligand.view()

Then, calculate its geometric center and use `Molecule.rotateBy` to rotate your ligand with the use of a rotation matrix `M`:

In [35]:
ligcenter = np.mean(ligand.get('coords'),axis=0)
M = uniformRandomRotation()
ligand.rotateBy(M,center=ligcenter)
ligand.view()

Another example is to rotate around an axis

In [36]:
ligand.rotateBy(rotationMatrix([1, 0, 0],math.pi),center= [0,0,0])

**Note:** the `uniformRandomRotation()` function provides the transformation matrix  for uniformly distributed random rotation.

## Working with trajectories

`Molecule` provides wrapping and aligning functionality for working with MD trajectories and improving the visualization.

In [37]:
molTraj = Molecule(htmd.home() + '/data/dhfr/dhfr.psf')
molTraj.read(htmd.home() + '/data/dhfr/dhfr.xtc')
molTraj.view()

## Exercise: find residues in contact with a ligand

Load the 'clean' molecule once again:

In [38]:
mol = Molecule('3PTB')

Identify residues in contact with the benzamidine ligand. We exploit the fact that there is exactly one C&alpha; (`name CA`) atom per residue.

In [39]:
mol.get("resid", sel="name CA and same residue as protein within 4 of resname BEN")

array([189, 190, 191, 192, 195, 213, 215, 216, 219, 220, 226])

## Exercise: find duplicate residues

Load the 'clean' molecule once again:

In [40]:
mol = Molecule('3PTB')

**Solution 1.** 

Identify duplicate residues, relying on PDB's *insertion* attribute:

In [41]:
# Quick and dirty
np.unique(mol.get('resid', sel='insertion A'))

array([184, 188, 221])

In [42]:
# Or equivantly, more explicit and pretty-printed
ia = mol.copy()
ia.filter("insertion A and name CA")
rid = ia.get('resid') # ia.resid also works!
rn = ia.get('resname')

for f, b in zip(rn, rid):
    print(f, b)

GLY 184
GLY 188
ALA 221


**Solution 2.**

If one doesn't want to rely on the *insertion* property, we can process the list of gaps between residues.

In [43]:
dups = mol.copy()                       # Make a working copy
dups.filter("name CA and protein")      # Keep only C-alphas
rid = dups.get('resid')                 # Get list of residue ids

nrid, count= np.unique(rid,return_counts=True)
                                        # Get how many times each residue id appeared
nrid[count>1]                           # Show duplicates

array([184, 188, 221])

## Exercise: find gaps in the sequence of residue numbers

We can exploit the `diff()` function to compute deltas between array elements.

In [44]:
# array with the "holes":
# - 0 means duplicate residues
# - >1 means segments of protein missing
deltas = np.diff(rid)
print(deltas)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 2
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 5 1 1 1 1 1 1 1 1 2 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


Print residue IDs at which the holes (including duplicate residues) occur:


In [45]:
new_rid = rid[:np.size(rid)-1] # no delta for last residue
new_rid[deltas!=1]

array([ 34,  67, 125, 130, 184, 188, 204, 217, 221])

A prettier and more explicit output:

In [46]:
# Iterate over all residues (excluding last one)
rn = dups.get('resname')
for i in range(np.size(rid)-1):
    # If there is a break...
    if(deltas[i]>1):
        # Remember that deltas[i]=rid[i+1]-rid[i]
        print(rid[i],rn[i],' followed by ',rid[i+1],rn[i+1])

34 ASN  followed by  37 SER
67 LEU  followed by  69 GLY
125 THR  followed by  127 SER
130 SER  followed by  132 ALA
204 LYS  followed by  209 LEU
217 SER  followed by  219 GLY
